<a href="https://colab.research.google.com/github/sunilkhadka139/LLM_Powered_AI_Framework_For_Semantic_Energy_Data_Search/blob/PeiYu/RAG_Transportation_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up

In [2]:
!pip install cohere==4.20.1
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge python-annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 825.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 11.6 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib_metadata 7.1.0
    Uninstalling importlib_metadata-7.1.0:
      Successfully uninstalled importlib_metadata-7.1.0
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...
Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - python-annoy


The following packages will 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Prepare for the Data

In [1]:
# Prepare for the data
from os import sep
import pandas as pd
data_path = "/content/drive/My Drive/Colab Notebooks/RAG/Data/Transportation_Data_Information.csv"
df = pd.read_csv(data_path,sep='|')

# Preprocessingc

#df['4emb']=str(df.to_dict(orient='records'))

#df['4emb']=df['Data Set'].str.replace('_',' ')+ ', ' + df['Description']
#df

In [41]:
total_sentences = []
for r in range(df.shape[0]):
  total_sentences.append(str((df.iloc[r].to_dict())))

total_sentences

["{'Data Set': 'Miovision_Intersection', 'Database': '[PCDOT_Miovision].[dbo].[mio_intersection_list]', 'Related Data Set': 'Miovision_Crosswalk', 'Description': 'The Miovision_Intersection dataset provides basic about individual intersections, offering identification numbers, user-assigned names, and geographic coordinates for location tracking. Additionally, the dataset allows for flexible data management through custom identifiers. More Info: [https://docs.api.miovision.com/]', 'Location to access data  (Directory / URL/ API)': 'API', 'Knowledge Base': 'Directory:(/Transportation_Data/Transportation_Miovision_Intersection.csv)', 'Example API ': 'https://catsapi.ua-smart.org/miovision/intersections', 'Constrain': nan}",
 '{\'Data Set\': \'Miovision_Crosswalk\', \'Database\': \'[PCDOT_Miovision].[dbo].[mio_crosswalk]\', \'Related Data Set\': \'Miovision_Intersection, Traffic_terms\', \'Description\': \'Miovision crosswalk data offers a concise summary of pedestrian and bicycle activit

# Sentence Embeddings

In [42]:
import cohere
from annoy import AnnoyIndex

co = cohere.Client('qYOIHoiJz1P3Jo7BeqdgxApGv0kTiCInhvGHW0MO')

#emb = co.embed(texts=df['4emb'].tolist(),
#               model='embed-english-v2.0').embeddings
emb = co.embed(texts=total_sentences,
               model='embed-english-v2.0').embeddings

# Explore the 10 first entries of the embeddings of the 3 sentences:
for e in emb:
    print(e[:3])

[-1.1181641, 0.25463867, 0.7495117]
[-1.1064453, 0.40966797, 0.44750977]
[-1.3388672, 1.5488281, 0.7871094]
[-0.05117798, -0.13061523, -0.08453369]
[-0.8725586, 0.014678955, 1.03125]


# Build a Search Index

In [43]:
import numpy as np

# Check the dimensions of the embeddings
embeds = np.array(emb)

# Create the search index, pass the size of embedding
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('test.ann')

True

# Searching most Relavant Data Set

In [ ]:
def search_dataset(query):
    # Get the query's embedding
    query_embed = co.embed(texts=[query],
                           model='embed-english-v2.0').embeddings

    # Retrieve the nearest neighbors
    similar_item_ids = search_index.get_nns_by_vector(query_embed[0],
                                                    10,
                                                  include_distances=True)

    search_results = df.iloc[similar_item_ids[0][0]]

    print('Data Set: ' + search_results['Data Set'])
    print('Related Data Set: ' + search_results['Related Data Set'])
    print(search_results['Description'])

    return search_results

In [54]:
def search_andrews_article(query):
    # Get the query's embedding
    query_embed = co.embed(texts=[query]).embeddings

    # Retrieve the nearest neighbors
    similar_item_ids = search_index.get_nns_by_vector(query_embed[0],
                                                    3,
                                                  include_distances=True)

    search_results = np.array(total_sentences)[similar_item_ids[0]]

    return search_results


# Print the top 10 relavent sentences with the question: "what is BEMcyclopedia?"
results = search_andrews_article(
    "Vehicle Speed of high density pedestrian area?"
)

print(results)

default model on embed will be deprecated in the future, please specify a model in the request.


['{\'Data Set\': \'Miovision_Crosswalk\', \'Database\': \'[PCDOT_Miovision].[dbo].[mio_crosswalk]\', \'Related Data Set\': \'Miovision_Intersection, Traffic_terms\', \'Description\': \'Miovision crosswalk data offers a concise summary of pedestrian and bicycle activities at intersections. Each entry is uniquely identified with a timestamp, recording the date and time of observations. The data categorizes movements into "Pedestrian" or "Bicycle," specifying the crosswalk side, direction (CW or CCW), and the quantity of observed activities. More Info: [https://docs.api.miovision.com/]\', \'Location to access data  (Directory / URL/ API)\': \'API\', \'Knowledge Base\': \'Directory:(/Transportation_Data/Transportation_Miovision_Crosswalk.csv)\', \'Example API \': \'https://catsapi.ua-smart.org/miovision/crosswalk?int_id=01560127-afd1-40e9-bd76-904340407fd8&int_id=0d4a1ca7-45e7-43cb-80b0-005d2f49303f&date=2024-01-01\', \'Constrain\': \'(1) Intersection ID and Date are required (2) Intersect

In [50]:
results = search_dataset(
    "Vehicle Speed of high density pedestrian area?"
)

NameError: name 'search_dataset' is not defined

In [51]:
def ask(question, num_generations=1):

    # Search the text archive
    results = search_andrews_article(question)

    # Get the top result
    context = results

    # Prepare the prompt
    prompt = f"""
    Excerpt from the BEMcyclopedia: https://bemcyclopedia.com/index.html:
    {context}
    Question: {question}

    Extract the answer of the question from the text provided.
    If the text doesn't contain the answer,
    reply that the answer is not available."""

#     print(prompt)

    prediction = co.generate(
        prompt=prompt,
        max_tokens=700,
        model="command-nightly",
        temperature=1,
        num_generations=num_generations
    )

    return prediction.generations

In [55]:
results = ask(
    "Vehicle Speed of high density pedestrian area?",
)

print(results[0])

default model on embed will be deprecated in the future, please specify a model in the request.


The answer is not available. The text does not provide any information about vehicle speeds in high-density pedestrian areas. It seems that the provided text is a description of various data sets and databases. If you have specific data sets or additional information, I can try to extract the answer for you.


In [56]:
results = ask(
    "what dataset should I take a look if I want to know the traffic speed?",
)

print(results[0])

default model on embed will be deprecated in the future, please specify a model in the request.


Inrix_Speed dataset.


In [1]:
results = ask(
    "what url should I take a look if I want to know the traffic speed?",
)

print(results[0])

NameError: name 'ask' is not defined

In [58]:
results = ask(
    "what url should I use if I want to know the traffic speed in segment id 12345?",
)

print(results[0])

default model on embed will be deprecated in the future, please specify a model in the request.


https://catsapi.ua-smart.org/inrix/speeds?segment_id=12345&int_id=null&date=current_date
